<a href="https://colab.research.google.com/github/takahashi0329/DeepLerarning2022Late/blob/main/%E5%8F%8E%E9%9B%86%E3%81%97%E3%81%9F%E7%94%BB%E5%83%8F%E3%81%A7%E5%AD%A6%E7%BF%92%E3%81%A8%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 収集した画像をColabにUPして学習させ、それをもとに分類させる

##ファイルのフォルダを準備する

In [7]:
import os
os.makedirs('sample', exist_ok=True) #分類対象の画像のフォルダを作成
os.makedirs('output/dog/sample', exist_ok=True) #犬と判定された時の置き場所
os.makedirs('output/cat/sample', exist_ok=True) #猫と判定された時の置き場所
os.makedirs('img/deep_learning/dog', exist_ok=True) #犬の学習用の収集画像
os.makedirs('img/deep_learning/cat', exist_ok=True) #猫の学習用の収集画像

## 必要なライブラリをimport

In [8]:
import cv2
import numpy as np
import glob as glob
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

##分類（クラス）の名前をフォルダ名（dog,cat）から引用する

In [9]:
path = "img/deep_learning" # dogとcatフォルダがある場所
folders = os.listdir(path)
# 2つのパスからフォルダを抜き出す
classes = [f for f in folders if os.path.isdir(os.path.join(path, f))]
print(classes)
n_classes = len(classes)
print(n_classes)

['cat', 'dog']
2


## 画像読み込みリサイズ、整形する

In [10]:
# 画像とラベルのための配列
X = [] #画像（学習データ用の配列）
Y = [] #ラベル（正解の配列）

# 画像を読み込みつつリサイズ
for label, class_name in enumerate(classes):
  files = glob.glob(path + "/" + class_name + "/*jpg")
  for file in files:
    img = cv2.imread(file)
    img = cv2.resize(img, dsize=(224,224))
    X.append(img)
    Y.append(label)
    # 内部ループ終わり
# 外部ループ終わり

## 0～255のビットデータ値を、学習のために0～1に変換(正規化)学習精度を上げる

In [11]:
X = np.array(X) # 配列XをNumpy配列に変換
X = X.astype('float32') #小数型に変換
X /=255.0
# ラベルもnumpyの配列に変換
Y = np.array(Y)
Y = np_utils.to_categorical(Y, n_classes)

### 整形したデータを学習用とテスト検証用にふりわける

In [12]:
# 学習データ8割、テスト検証用が2割にわける
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
# 学習データ8割
print(X_train.shape)
# テストデータ2割
print(X_test.shape)
# 学習データ8割(正解ラベル)
print(Y_train.shape)
# テストデータ2割(正解ラベル)
print(Y_test.shape)

(120, 224, 224, 3)
(31, 224, 224, 3)
(120, 2)
(31, 2)


## 準備したデータを学習させていく

###学習に必要なライブラリをimport

In [13]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Activation, Dense, Flatten, Dropout
from keras.optimizers import Adam

##モデルのVGG16をアレンジしていく

In [14]:
# VGG16 入力層を用意する（定義する）
input_tensor = Input(shape=(224, 224, 3))
# VGG16のインスタンスを作る（最後のsofmaxの1000層を無効にして省く）
base_model = VGG16(weights='imagenet', input_tensor=input_tensor, include_top=False)

58889256/58889256 [==============================] - 3s 0us/step


## 1000分類のsoftmaxを外した代わりに、2つに分類するsoftmax層を追加する

In [15]:
# 追加する層の作成
top_model = Sequential() #ラッピングする層
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(n_classes, activation='softmax'))

###最後のSOFTMAXを外したモデル(basemodel)と追加用の2つに分類するSOFTMAX層のモデル(top_model)を繋げる

In [16]:
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

###時間短縮のために、15層まで外す

In [17]:
#ループしながら15層までを無効にする
for layer in model.layers[:15]:
  layer.trainable = False #15層までは学習しない
#ループ終わり
print('# laers=', len(model.layers)) #モデルの層を表示

# laers= 20


##学習モデルをコンパイルする

In [18]:
# 損失はクロスエントロピー法で算出、最適化はADAM、指標は精度でみる
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [19]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## コンパイルしたモデルで収集した画像データを学習させる

In [20]:
model.fit(X_train,Y_train,epochs=20,batch_size=16)

Epoch 1/20
8/8 [==============================] - 13s 249ms/step - loss: 1.2520 - accuracy: 0.5333
Epoch 2/20
8/8 [==============================] - 1s 91ms/step - loss: 0.8785 - accuracy: 0.5417
Epoch 3/20
8/8 [==============================] - 1s 96ms/step - loss: 0.5642 - accuracy: 0.6917
Epoch 4/20
8/8 [==============================] - 1s 89ms/step - loss: 0.6899 - accuracy: 0.5333
Epoch 5/20
8/8 [==============================] - 1s 98ms/step - loss: 0.6846 - accuracy: 0.5750
Epoch 6/20
8/8 [==============================] - 1s 94ms/step - loss: 0.6883 - accuracy: 0.6083
Epoch 7/20
8/8 [==============================] - 1s 95ms/step - loss: 0.6240 - accuracy: 0.7167
Epoch 8/20
8/8 [==============================] - 1s 90ms/step - loss: 0.5546 - accuracy: 0.8000
Epoch 9/20
8/8 [==============================] - 1s 92ms/step - loss: 0.4091 - accuracy: 0.8583
Epoch 10/20
8/8 [==============================] - 1s 88ms/step - loss: 0.3468 - accuracy: 0.8750
Epoch 11/20
8/8 [==========

##学習の成果を指標(accuracy)で確認

In [21]:
score = model.evaluate(X_test, Y_test, batch_size=16)

2/2 [==============================] - 2s 2s/step - loss: 2.8237 - accuracy: 0.6452


### 作成した学習モデルとクラス分類名を保存

In [23]:
import pickle
# vクラス分類の保存
pickle.dump(classes, open('classes.sav', 'wb')) # オブジェクトをバイナリファイルで保存
# 学習モデル保存
model.save('cnn.h5')

##保存したクラス分類と学習モデルを読み込んで使う

In [24]:
# ライブラリのimport
from keras.models import load_model
import pickle
import cv2
import glob

##ファイルを読み込んでオブジェクトとしてよみがえらせる